<a href="https://colab.research.google.com/github/ghostfm3/ksks/blob/master/jpwordnet1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sqlite3

In [2]:
conn = sqlite3.connect("wnjpn.db")

In [3]:
cur = conn.execute("select name from sqlite_master where type='table'")
for row in cur:
    print(row)

('pos_def',)
('link_def',)
('synset_def',)
('synset_ex',)
('synset',)
('synlink',)
('ancestor',)
('sense',)
('word',)
('variant',)
('xlink',)


In [4]:
cur = conn.execute("select * from word limit 10")
for row in cur:
    print(row)

(1, 'eng', 'expletive', None, 'n')
(2, 'eng', 'measles', None, 'n')
(3, 'eng', 'contras', None, 'n')
(4, 'eng', 'beef_man', None, 'n')
(5, 'eng', 'dwelling', None, 'n')
(6, 'eng', 'acetum', None, 'n')
(7, 'eng', 'mount_carmel', None, 'n')
(8, 'eng', 'class_taxopsida', None, 'n')
(9, 'eng', 'vascular_ray', None, 'n')
(10, 'eng', 'genus_salamandra', None, 'n')


In [5]:
cur = conn.execute("PRAGMA TABLE_INFO(word)")
for row in cur:
    print(row)

(0, 'wordid', 'integer', 0, None, 1)
(1, 'lang', 'text', 0, None, 0)
(2, 'lemma', 'text', 0, None, 0)
(3, 'pron', 'text', 0, None, 0)
(4, 'pos', 'text', 0, None, 0)


In [6]:
cur = conn.execute("select count(*) from word")
for row in cur:
    print("Wordnetに登録されているWordDBの単語数：%s" % row[0])

Wordnetに登録されているWordDBの単語数：249121


In [7]:
# 特定の単語を入力とした時に、類義語を検索する関数
def SearchSimilarWords(word):

    # 問い合わせしたい単語がWordnetに存在するか確認する
    cur = conn.execute("select wordid from word where lemma='%s'" % word)
    word_id = 99999999  #temp 
    for row in cur:
        word_id = row[0]

    # Wordnetに存在する語であるかの判定
    if word_id==99999999:
        print("「%s」は、Wordnetに存在しない単語です。" % word)
        return
    else:
        print("【「%s」の類似語を出力します】\n" % word)

    # 入力された単語を含む概念を検索する
    cur = conn.execute("select synset from sense where wordid='%s'" % word_id)
    synsets = []
    for row in cur:
        synsets.append(row[0])

    # 概念に含まれる単語を検索して画面出力する
    no = 1
    for synset in synsets:
        cur1 = conn.execute("select name from synset where synset='%s'" % synset)
        for row1 in cur1:
            print("%sつめの概念 : %s" %(no, row1[0]))
        cur2 = conn.execute("select def from synset_def where (synset='%s' and lang='jpn')" % synset)
        sub_no = 1
        for row2 in cur2:
            print("意味%s : %s" %(sub_no, row2[0]))
            sub_no += 1
        cur3 = conn.execute("select wordid from sense where (synset='%s' and wordid!=%s)" % (synset,word_id))
        sub_no = 1
        for row3 in cur3:
            target_word_id = row3[0]
            cur3_1 = conn.execute("select lemma from word where wordid=%s" % target_word_id)
            for row3_1 in cur3_1:
                print("類義語%s : %s" % (sub_no, row3_1[0]))
                sub_no += 1
        print("\n")
        no += 1

In [8]:
SearchSimilarWords("ぼんやり")

【「ぼんやり」の類似語を出力します】

1つめの概念 : mistily
意味1 : 漠然とした方法で
類義語1 : mistily
類義語2 : vaguely
類義語3 : ぼうっと
類義語4 : 何が無し
類義語5 : 茫と
類義語6 : 何と無く
類義語7 : 薄々
類義語8 : うすうす
類義語9 : ぼやっと
類義語10 : 何となく
類義語11 : なんとなく
類義語12 : 薄薄
類義語13 : なにがなし
類義語14 : 何処となく
類義語15 : 杳として
類義語16 : 何処と無く
類義語17 : もやもや
類義語18 : 何がなし


2つめの概念 : vacantly
意味1 : 空虚な態度で
類義語1 : vacantly


3つめの概念 : absentmindedly
意味1 : ぼんやりした、あるいは気を取られている態度で
類義語1 : absentmindedly
類義語2 : abstractedly
類義語3 : absently
類義語4 : inattentively
類義語5 : ぽかんと
類義語6 : うかうか
類義語7 : あっけらかん


